In [1]:
%cd /home/slav/ai/claim-rank
!export PYTHONPATH=.
%load_ext autoreload
%autoreload 2

/home/slav/ai/claim-rank


In [2]:
import numpy as np
np.random.seed(42) # ! before importing keras!

In [3]:
from src.data.debates import read_debates, Debate

In [4]:
from src.features.feature_sets import get_cb_pipeline

In [5]:
from keras.layers import Input, Embedding, Conv1D, Dense
from keras.models import Model, clone_model
from keras.layers import Input, Dense, Dropout
from keras import optimizers
from src.stats.rank_metrics import average_precision, precision_at_n, accuracy_rounded
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing.data import MinMaxScaler

Using TensorFlow backend.


In [6]:
train_sentences = read_debates(Debate.FIRST) + read_debates(Debate.VP)
val_sentences = read_debates(Debate.SECOND)
test_sentences = read_debates(Debate.THIRD)

pipeline = get_cb_pipeline(train=train_sentences)
X_train = pipeline.fit_transform(train_sentences)
X_val = pipeline.fit_transform(val_sentences)
X_test = pipeline.transform(test_sentences)

y_train_pf = np.array([int(s.labels[5]) for s in train_sentences])
y_train_all = np.array([1 if s.label>0 else 0 for s in train_sentences])

y_val_pf = np.array([int(s.labels[5]) for s in val_sentences])
y_val_all = np.array([1 if s.label>0 else 0 for s in val_sentences])

y_test_pf = np.array([int(s.labels[5]) for s in test_sentences])
y_test_all = np.array([1 if s.label>0 else 0 for s in test_sentences])


y_train_0 = np.array([int(s.labels[0]) for s in train_sentences])
y_train_1 = np.array([int(s.labels[1]) for s in train_sentences])
y_train_2 = np.array([int(s.labels[2]) for s in train_sentences])
y_train_3 = np.array([int(s.labels[3]) for s in train_sentences])
y_train_4 = np.array([int(s.labels[4]) for s in train_sentences])
y_train_5 = np.array([int(s.labels[5]) for s in train_sentences])
y_train_6 = np.array([int(s.labels[6]) for s in train_sentences])

['tokens_num', 'text_len', 'bag_tfidf', 'pos']


In [21]:
print(X_train.shape)
print(train_sentences[103].labels)
print(train_sentences[103].label)
print(X_train.shape[1])
print(np.array(y_train_all).shape)
print(y_val_pf[13])
print(y_val_all[13])

(2761, 1363)
['0', '0', '1', '0', '0', '0', '0', '0', '0']
1
1363
(2761,)
1
1


In [22]:
features_count = X_train.shape[1]

In [33]:
def create_baseline_model(out_count):
    input_layer = Input(shape=(features_count,))
    x = Dense(features_count, kernel_initializer='normal', activation='relu')(input_layer)
    
    outputs = list(map(lambda _: Dense(1, kernel_initializer='normal', activation='sigmoid')(x), range(out_count)))

    model = Model(inputs=[input_layer], outputs=outputs)
                         
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model;

In [41]:
def run_single_model(train_target, label_func, iters = 20, epochs=5):
    model = create_baseline_model(1);

    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;

    for epoch in range(iters):
        
        model.fit(X_train, y=[train_target], epochs=epochs, verbose=0, batch_size=550)

        predicted_val = model.predict(X_val)
        
        for pred, sentence in zip(predicted_val, val_sentences):
            sentence.pred = pred[0]

        av_p = average_precision(val_sentences,1, label_func)
        
        print(str(epoch) + ':' + str(av_p) + ' ', end='')
        
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;
    
    print()

    print(best_av_p)
    print((best_epoch + 1) * epochs)

    predicted_val = best_model.predict(X_test)
    for pred, sentence in zip(predicted_val, test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1,label_func))
    
    return [best_model, best_av_p, best_epoch]
    

In [44]:
def run_multiple_model(train_targets, iters = 20, epochs=5):
    model = create_baseline_model(len(train_targets));

    best_model = None;
    best_av_p = 0.;
    best_epoch = -1;

    best_model_all = None;
    best_av_p_all = 0.;
    best_epoch_all = -1;

    for epoch in range(iters):
        model.fit(X_train, y=train_targets, epochs=epochs, verbose=0, batch_size=550)

        predicted_val = model.predict(X_val)
        
        for pred, sentence in zip(predicted_val[1], val_sentences):
            sentence.pred = pred[0]

        av_p = average_precision(val_sentences,1, lambda x: x.labels[5])
        
        print(str(epoch) + ':' + str(av_p) + ' ', end='')
        
        if (av_p > best_av_p):
            best_model = clone_model(model);
            best_av_p = av_p;
            best_epoch = epoch;

        for pred, sentence in zip(predicted_val[0], val_sentences):
            sentence.pred = pred[0]

        av_p_all = average_precision(val_sentences,1)
        if (av_p_all > best_av_p_all):
            best_model_all = clone_model(model);
            best_av_p_all = av_p_all;
            best_epoch_all = epoch;
    
    print()
    
    print('All:')
    print(best_av_p_all)
    print((best_epoch_all+ 1) * epochs)

    predicted_val = best_model_all.predict(X_test)
    for pred, sentence in zip(predicted_val[0], test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1))

    print('Pf:')
    print(best_av_p)
    print((best_epoch + 1) * epochs)

    predicted_val = best_model.predict(X_test)
    for pred, sentence in zip(predicted_val[1], test_sentences):
        sentence.pred = pred[0]

    print(average_precision(test_sentences,1,lambda x: x.labels[5]))
    
    return [[best_model, best_av_p, best_epoch], [best_model_all, best_av_p_all, best_epoch_all]]
    

In [37]:
run_single_model(y_train_pf, lambda x: x.labels[5])

0:0.08568362776319396 1:0.0841058544893181 2:0.08327489554230191 3:0.08465425209502174 4:0.08520665391177969 5:0.08558767514325119 6:0.0858612471152793 7:0.08608524650149238 8:0.08477297324612136 9:0.08456764089679783 10:0.08489820980096813 11:0.08572801403180297 12:0.0837582979655827 13:0.0849257061782979 14:0.08430671644968105 15:0.08350463210940727 16:0.08388878088547444 17:0.08327481061547667 18:0.08440377336356614 19:0.08502413122360596 
0.08608524650149238
40
0.03760844293588806


[<keras.engine.training.Model at 0x7f65090ede48>, 0.08608524650149238, 7]

In [43]:
run_multiple_model([y_train_all, y_train_pf])

0 0:0.04049282159721163 1 1:0.06176059174088208 2 2:0.07245641306015117 3 3:0.0796399638736438 4 4:0.08254168387207181 5 5:0.07764345619829341 6 6:0.07782296228799981 7 7:0.08029625947806485 8 8:0.08009007491456578 9 9:0.08002146038208462 10 10:0.08072669389409588 11 11:0.07954248148944153 12 12:0.08157952872338953 13 13:0.08003625558159179 14 14:0.07679120993505488 15 15:0.07567183805929156 16 16:0.07955342810216454 17 17:0.07907411439449379 18 18:0.07900856703621333 19 19:0.07918662014610028 
All:
0.2873262760389052
10
0.2635333672595357
Pf:
0.08254168387207181
25
0.03517072207960124


[[<keras.engine.training.Model at 0x7f64f3cd0668>, 0.08254168387207181, 4],
 [<keras.engine.training.Model at 0x7f64f3dea748>, 0.2873262760389052, 1]]